## Similaridade de Strings

Aplicações:
    * Data Cleaning
    * Correção de digitação
    * Tradução de idiomas

FuzzyWuzzy - Levenshtein_distance - https://en.wikipedia.org/wiki/Levenshtein_distance

**Instalando a biblioteca Fuzzywuzzy**

#Caso não tenha instalado troque para code e execute.
!pip install fuzzywuzzy

#Essa é uma segunda opção de instalação.
!pip install fuzzywuzzy[speedup]

**Importa a biblitoteca**

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

**Aplicando a fuzzywuzzy em duas strings**

In [4]:
fuzz.ratio('São Paulo', 'São Paul')

94

94 seria a porcentagem da similariadade entre as duas string

In [5]:
fuzz.ratio('São Paulo', 'São Paulo')

100

In [6]:
s1 = 'Belo Horizonte'
s2 = 'B. Horizonte'
fuzz.ratio(s1,s2)

85

**Letras maiusculas e minúsculas**

In [7]:
s1 = 'São Paulo'
s2 = 'são paulo'
fuzz.ratio(s1,s2)

78

** Pontuação ou outros caracteres influenciam no score**

In [8]:
s1 = 'São Paulo'
s2 = 'São Paulo!!'
fuzz.ratio(s1,s2)

90

## Similaridade Parcial

* Similaridade parcial busca apenas a string em questão e descarta o resto.
* Extremamente útil para trabalhar com dados coletados da web ou ainda quando queremos ignorar pontuações.

In [9]:
# Consultando o score usando o método ratio
s1 = 'São Paulo'
s2 = '###$$%$!São Paulo#$#%#ˆˆˆˆˆ!!'
fuzz.ratio(s1,s2)

47

In [10]:
# Consultando o score usando o método partial
s1 = 'São Paulo'
s2 = '###$$%$!São Paulo#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1,s2)

100

O metodo partial ignora os caracteres especiais

In [12]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'São Paulo'
s2 = '###$$%$!São Paullo#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1,s2)

89

** Ordem de caracteres diferentes?**

In [14]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'São Paulo'
s2 = 'Paulo São'
fuzz.partial_ratio(s1,s2)

56

* Função **partial_token_sort_ratio()** separa os tokens por espaço e ordena por ordem alfabética.
* Coloca as strings em letras minúsculas.
* Considera apenas as strings consultadas.

In [15]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'São Paulo'
s2 = 'Paulo São'
fuzz.partial_token_sort_ratio(s1,s2)

100

In [16]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'São Paulo'
s2 = 'São Paullo'
fuzz.partial_token_sort_ratio(s1,s2)

88

In [17]:
# Consultando o score usando o método partial e com caracteres minusculos.
s1 = 'São Paulo'
s2 = '###$$%$!são paulo#$#%#ˆˆˆˆˆ!!'
fuzz.partial_token_sort_ratio(s1,s2)

100

## Processando uma Lista de Strings

* Aplicar o fuzzywuzzy para corrigir strings em uma base de dados

In [18]:
from fuzzywuzzy import process

** Cria lista de strings**

In [19]:
lista = ['Doença Cardiovascular.','doença cardiovascular!!', 'Doenca Cardiovascular', 'Doenc. Cardio']

In [20]:
lista

['Doença Cardiovascular.',
 'doença cardiovascular!!',
 'Doenca Cardiovascular',
 'Doenc. Cardio']

** Extrai os scores de similaridades com uma string em questão**

In [21]:
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio)

[('Doença Cardiovascular.', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95),
 ('Doenc. Cardio', 85)]

** Limitando o retorno**

In [22]:
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio, limit=2)

[('Doença Cardiovascular.', 100), ('doença cardiovascular!!', 100)]

** Retorna apenas uma string com um score acima de 95**

In [23]:
process.extractOne('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio, score_cutoff=95)

('Doença Cardiovascular.', 100)

## Data Cleaning em um DataFrame

* Aplicar o fuzzywuzzy em uma base de dados
* Medir a similaridade de strings e fazer **Data Cleaning**

In [24]:
import pandas as pd
from collections import OrderedDict
data = OrderedDict(
    {
        'descrição': ['São Paulo', 'SãoPaulo', 'São Pauloo','São Paulo,,', 'Belo Horizonte', 'B. Horizonte']
    })

/home/dieinimy/.pyenv/versions/3.7.4/envs/jupyter3/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [25]:
data

OrderedDict([('descrição',
              ['São Paulo',
               'SãoPaulo',
               'São Pauloo',
               'São Paulo,,',
               'Belo Horizonte',
               'B. Horizonte'])])

**Converte Dicionário para Pandas Dataframe**

In [26]:
df = pd.DataFrame(data)

In [27]:
df

,descrição
0,São Paulo
1,SãoPaulo
2,São Pauloo
3,"São Paulo,,"
4,Belo Horizonte
5,B. Horizonte


**Corrigindo dados do dataframe**

Para que seja possivel fazer as comparações é necessário ter o valor correto de cadas string.

Por esse motivo é criado uma lista com as palavras corretas para comparação


In [35]:
lista_cidades = ['Belo Horizonte', 'São Paulo']

In [36]:
lista_cidades

['Belo Horizonte', 'São Paulo']

**Mostrando a porcentagem da similariadade**

In [30]:
for cidade in lista_cidades:
    for i in df.descrição.items():
        print ('{}, {}, Similaridade: {}'.format(cidade, i[1], fuzz.partial_token_sort_ratio(cidade,i)))

Belo Horizonte, São Paulo, Similaridade: 30
Belo Horizonte, SãoPaulo, Similaridade: 33
Belo Horizonte, São Pauloo, Similaridade: 27
Belo Horizonte, São Paulo,,, Similaridade: 30
Belo Horizonte, Belo Horizonte, Similaridade: 100
Belo Horizonte, B. Horizonte, Similaridade: 77
São Paulo, São Paulo, Similaridade: 100
São Paulo, SãoPaulo, Similaridade: 77
São Paulo, São Pauloo, Similaridade: 88
São Paulo, São Paulo,,, Similaridade: 100
São Paulo, Belo Horizonte, Similaridade: 50
São Paulo, B. Horizonte, Similaridade: 25


**Atualizando as linhas do Dataframe se similaridade for maior que um determinado valor**
Neste caso utilizamos **70%** tendo em vista o resultado anterior.

Acho interessante tentar criar algo que facilite essa visualização para detecção do melhor percentual para correção da string.

In [33]:
for cidade in lista_cidades:
    for i in df.descrição.items():
        if fuzz.partial_token_sort_ratio(cidade,i[1]) >= 70:
            df.loc[df['descrição'] == i[1], ['descrição']] = cidade

**Feita a correção vamos visualizar como ficou o Dataframe**

In [37]:
df

,descrição
0,São Paulo
1,São Paulo
2,São Paulo
3,São Paulo
4,Belo Horizonte
5,Belo Horizonte
